In [5]:
import requests
import zipfile
import os

workspace = r'C:\School\GIS 5572\Local\Lab 2\data'

land_cover_resource = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/fgdb_elev_30m_digital_elevation_model.zip'

response = requests.get(land_cover_resource)
file_path = os.path.join(workspace, 'elevation.zip')
with open(file_path, 'wb') as file:
    file.write(response.content)
        
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(workspace)

In [83]:
raster_path = os.path.join(workspace, 'elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m')
desc = arcpy.Describe(raster_path)

# Expected bounding box values
expected_xmin = 189775.332039
expected_ymin = 4816305.370038
expected_xmax = 761665.332039
expected_ymax = 5472435.370038

# Expected spatial reference name
expected_spatial_reference = 'NAD_1983_UTM_Zone_15N'

# Expexted cell size
expected_Width = 30.0
expected_Height = 30.0

# Print the bounding box
print("Bounding Box:")
print("XMin: {0}, YMin: {1}, XMax: {2}, YMax: {3}".format(desc.extent.XMin, desc.extent.YMin, desc.extent.XMax, desc.extent.YMax))

# Print the spatial reference
print("\nSpatial Reference:")
print(desc.spatialReference.name)

#Print the cell size
print(f"Cell size (X): {desc.meanCellWidth}")
print(f"Cell size (Y): {desc.meanCellHeight}")

# Assert statements to check the values
assert desc.extent.XMin == expected_xmin, f"XMin does not match. Expected: {expected_xmin}, Actual: {desc.extent.XMin}"
assert desc.extent.YMin == expected_ymin, f"YMin does not match. Expected: {expected_ymin}, Actual: {desc.extent.YMin}"
assert desc.extent.XMax == expected_xmax, f"XMax does not match. Expected: {expected_xmax}, Actual: {desc.extent.XMax}"
assert desc.extent.YMax == expected_ymax, f"YMax does not match. Expected: {expected_ymax}, Actual: {desc.extent.YMax}"
assert desc.spatialReference.name == expected_spatial_reference, f"Spatial reference does not match. Expected: {expected_spatial_reference}, Actual: {desc.spatialReference.name}"
assert desc.meanCellWidth == expected_Width, f"Spatial reference does not match. Expected: {expected_Width}, Actual: {desc.meanCellWidth}"
assert desc.meanCellHeight == expected_Height, f"Spatial reference does not match. Expected: {expected_Height}, Actual: {desc.meanCellHeight}"

print("\nAll values match the expected values.")

Bounding Box:
XMin: 189775.332039, YMin: 4816305.370038, XMax: 761665.332039, YMax: 5472435.370038

Spatial Reference:
NAD_1983_UTM_Zone_15N
Cell size (X): 30.0
Cell size (Y): 30.0

All values match the expected values.


In [84]:
raster = arcpy.Raster(raster_path)

min_value = raster.minimum
max_value = raster.maximum


# Check if the range is between 590 and 2300
if min_value >= 590 and max_value <= 2301:
    print("Data range is between 0 and 2301.")
else:
    print("Data range is not between 590 and 2301.")

Data range is between 0 and 2301.


In [85]:
raster_out = r'C:\School\GIS 5572\Local\Lab 2\data\coarse_raster'
arcpy.sa.Aggregate(raster_path, 100, "MEAN").save(raster_out)

In [86]:
arcpy.env.workspace = r'C:\School\GIS 5572\Local\Lab 1.2\Test Project\PostgreSQL-35-gis5572(postgres).sde'
spatial_ref = arcpy.SpatialReference(26915)
out_name = "elevation_points"
geometry_type = "POINT"
arcpy.CreateFeatureclass_management(arcpy.env.workspace, out_name, geometry_type, spatial_reference = spatial_ref)

<Result 'C:\\School\\GIS 5572\\Local\\Lab 1.2\\Test Project\\PostgreSQL-35-gis5572(postgres).sde\\elevation_points'>

In [87]:
out_name = r'C:\School\GIS 5572\Local\Lab 1.2\Test Project\PostgreSQL-35-gis5572(postgres).sde\gis5572.postgres.elevation_points'
arcpy.RasterToPoint_conversion(raster_out, out_name, "VALUE")

<Result 'C:\\School\\GIS 5572\\Local\\Lab 1.2\\Test Project\\PostgreSQL-35-gis5572(postgres).sde\\gis5572.postgres.elevation_points'>